In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference_replisome.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████████████████████████| 30/30 [00:12<00:00,  2.39it/s]


eval images shape: (30, 1041, 1302) contain 20937 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.00625,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.4,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 115.0,
 'e_per_adu': 0.4,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/RN_e_venus.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/replisome_dots/pool_3024/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/replisome_dots/pool_3024/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6}

In [10]:
start_time = time.time()
model.fit(batch_size=32, max_iters=20000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 1355.566 || Time Upd.: 489.5 ms  || BatchNr.: 100
 || Cost: 15438.735 || Time Upd.: 472.7 ms  || BatchNr.: 200
 || Cost: 470.582 || Time Upd.: 469.6 ms  || BatchNr.: 300
 || Cost: 316.688 || Time Upd.: 468.8 ms  || BatchNr.: 400
 || Cost: 244.481 || Time Upd.: 468.0 ms  || BatchNr.: 500
 || Cost: 223.876 || Time Upd.: 462.1 ms  || BatchNr.: 600
 || Cost: 210.621 || Time Upd.: 462.0 ms  || BatchNr.: 700
 || Cost: 199.554 || Time Upd.: 467.1 ms  || BatchNr.: 800
 || Cost: 182.958 || Time Upd.: 462.0 ms  || BatchNr.: 900
 || Cost: 159.017 || Time Upd.: 458.9 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:12068
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 12068
JoR: 0.799 || Eff_3d: 30.777 || Jaccard: 0.572 || Factor: 805.933 || RMSE_lat: 71.536 || RMSE_ax: 

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:14689
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 14689
JoR: 1.162 || Eff_3d: 45.990 || Jaccard: 0.685 || Factor: 744.626 || RMSE_lat: 58.898 || RMSE_ax: 53.043 || RMSE_x: 41.735 || RMSE_y: 41.559 || Cost: 102.013 || Recall: 0.692 || Precision: 0.986 || BatchNr.: 2300 || Cost: 102.013 || Time Upd.: 461.0 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 45.990 || 58.898 || 53.043 || 2300


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:7899
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 7899
JoR: 0.754 || Eff_3d: 25.911 || Jaccard: 0.377 || Factor: 526.498 || RMSE_lat:

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:10954
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 10954
JoR: 1.027 || Eff_3d: 38.635 || Jaccard: 0.523 || Factor: 614.662 || RMSE_lat: 50.936 || RMSE_ax: 46.046 || RMSE_x: 36.230 || RMSE_y: 35.803 || Cost: 90.012 || Recall: 0.523 || Precision: 1.000 || BatchNr.: 3600 || Cost: 90.012 || Time Upd.: 595.9 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 38.635 || 50.936 || 46.046 || 3600


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:13193
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 13193
JoR: 1.176 || Eff_3d: 45.541 || Jaccard: 0.630 || Factor: 660.591 || RMSE_lat:

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:13661
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 13661
JoR: 1.195 || Eff_3d: 47.323 || Jaccard: 0.651 || Factor: 685.956 || RMSE_lat: 54.470 || RMSE_ax: 41.593 || RMSE_x: 38.248 || RMSE_y: 38.783 || Cost: 81.632 || Recall: 0.651 || Precision: 0.998 || BatchNr.: 4900 || Cost: 81.632 || Time Upd.: 492.9 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 47.323 || 54.470 || 41.593 || 4900


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:12211
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 12211
JoR: 1.102 || Eff_3d: 43.068 || Jaccard: 0.583 || Factor: 603.935 || RMSE_lat:

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:14507
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 14507
JoR: 1.279 || Eff_3d: 50.801 || Jaccard: 0.692 || Factor: 657.969 || RMSE_lat: 54.112 || RMSE_ax: 37.875 || RMSE_x: 38.478 || RMSE_y: 38.047 || Cost: 73.614 || Recall: 0.692 || Precision: 0.999 || BatchNr.: 6200 || Cost: 73.614 || Time Upd.: 505.9 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 50.801 || 54.112 || 37.875 || 6200


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:14652
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 14652
JoR: 1.262 || Eff_3d: 50.512 || Jaccard: 0.699 || Factor: 693.567 || RMSE_lat:

...

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:15569
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 15569
JoR: 1.371 || Eff_3d: 54.102 || Jaccard: 0.743 || Factor: 674.503 || RMSE_lat: 54.189 || RMSE_ax: 37.578 || RMSE_x: 39.302 || RMSE_y: 37.306 || Cost: 65.962 || Recall: 0.743 || Precision: 1.000 || BatchNr.: 14000 || Cost: 65.962 || Time Upd.: 473.6 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 54.102 || 54.189 || 37.578 || 14000


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:16311
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 16311
JoR: 1.405 || Eff_3d: 55.621 || Jaccard: 0.778 || Factor: 731.691 || RMSE_la

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:16578
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 16578
JoR: 1.417 || Eff_3d: 55.884 || Jaccard: 0.791 || Factor: 716.684 || RMSE_lat: 55.808 || RMSE_ax: 39.060 || RMSE_x: 40.964 || RMSE_y: 37.901 || Cost: 66.255 || Recall: 0.791 || Precision: 0.999 || BatchNr.: 15300 || Cost: 66.255 || Time Upd.: 517.0 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 55.884 || 55.808 || 39.060 || 15300


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:15685
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 15685
JoR: 1.382 || Eff_3d: 54.484 || Jaccard: 0.749 || Factor: 664.496 || RMSE_la

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:16097
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 16097
JoR: 1.387 || Eff_3d: 54.548 || Jaccard: 0.768 || Factor: 685.645 || RMSE_lat: 55.414 || RMSE_ax: 40.720 || RMSE_x: 40.642 || RMSE_y: 37.668 || Cost: 64.114 || Recall: 0.769 || Precision: 1.000 || BatchNr.: 16600 || Cost: 64.114 || Time Upd.: 458.7 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 54.548 || 55.414 || 40.720 || 16600


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:15798
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 15798
JoR: 1.345 || Eff_3d: 53.942 || Jaccard: 0.754 || Factor: 671.940 || RMSE_la

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:15431
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 15431
JoR: 1.365 || Eff_3d: 53.782 || Jaccard: 0.736 || Factor: 655.636 || RMSE_lat: 53.940 || RMSE_ax: 37.697 || RMSE_x: 39.085 || RMSE_y: 37.174 || Cost: 65.380 || Recall: 0.737 || Precision: 1.000 || BatchNr.: 17900 || Cost: 65.380 || Time Upd.: 512.0 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 53.782 || 53.940 || 37.697 || 17900


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:16726
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 16726
JoR: 1.449 || Eff_3d: 56.812 || Jaccard: 0.798 || Factor: 714.018 || RMSE_la

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:16670
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 16670
JoR: 1.448 || Eff_3d: 56.921 || Jaccard: 0.796 || Factor: 714.295 || RMSE_lat: 54.961 || RMSE_ax: 36.851 || RMSE_x: 40.401 || RMSE_y: 37.262 || Cost: 64.837 || Recall: 0.796 || Precision: 1.000 || BatchNr.: 19200 || Cost: 64.837 || Time Upd.: 458.3 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 56.921 || 54.961 || 36.851 || 19200


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 20937, preds:15535
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 20937 ,preds: 15535
JoR: 1.368 || Eff_3d: 53.990 || Jaccard: 0.742 || Factor: 654.587 || RMSE_la